# Quantifying solidification of metallic alloys with scikit-image

<center><h2>C. Gus Becker</h2></center>
<center>
    <img src="images/csm_logo.png" width="400">
    <img src="images/canfsa_logo.png" width="300">
</center>
<center><h3>&</h3></center>
<center><h2>Marianne Corvellec</h2></center>
<center>
    <img src="images/skimage_logo.png" width="300">
    <img src="images/igdore_logo.png" width="300">
</center>

#### BIDS ImageXD Conference — May 18, 2021 ― Session 4: Materials Science

# Context and Motivation
<img src="images/melt_pool.jpg">
<h4><a href="https://commons.wikimedia.org/wiki/File:Selective_laser_melting_system_schematic.jpg">Materialgeeza</a>, <a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA 3.0</a> (Wikimedia Commons)</h4>

# Introduction to scikit-image
<br />
<img src="images/skimage_logo.png">
<br />
https://scikit-image.org/

In [ ]:
import skimage

In [ ]:
from skimage import data

In [ ]:
cat = data.cat()

In [ ]:
type(cat)

In [ ]:
cat.ndim

In [ ]:
import plotly.express as px

In [ ]:
px.imshow(cat)

In [ ]:
retina = data.retina()

In [ ]:
px.imshow(retina)

In [ ]:
from skimage import color, filters

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

image = retina[:, :, 2]

thresh = filters.threshold_multiotsu(image)
regions = np.digitize(image, bins=thresh)

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(image)
ax[0].set_title('Channel 2 in RGB')
ax[1].imshow(regions)
ax[1].set_title('Multi-Otsu thresholding')
plt.show()

In [ ]:
conv_img = color.rgb2lab(retina)
image = conv_img[:, :, 1]

thresh = filters.threshold_multiotsu(image)
regions = np.digitize(image, bins=thresh)

fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(image)
ax[0].set_title('Channel 1 in CIELAB')
ax[1].imshow(regions)
ax[1].set_title('Multi-Otsu thresholding')
plt.show()

# Track Solid-Liquid Interface

In [ ]:
import imageio

collection = []
reader = imageio.get_reader('data/nickel_solidification.tif')
for img in reader:
    collection.append(img)
    
len(collection)

In [ ]:
collection[0].shape

In [ ]:
full_frames = np.array(collection)
full_frames.shape

In [ ]:
fig = px.imshow(full_frames, animation_frame=0, binary_string=True)
fig.show()

In [ ]:
frames = np.stack([img[60:110, 175:250] for img in collection])
frames.shape

In [ ]:
fig = px.imshow(frames, animation_frame=0, binary_string=True)
fig.show()

# Import local modules containing plotting and processing functions

In [ ]:
# Import plotting module from local package
from solidification import plot

# Show raw images

In [ ]:
fig, axes = plot.plot_imgs(
    [frames[1], frames[4], frames[7]],
    row_title='Raw Images'
)

# Apply Gaussian filter and subtract current image from succeeding image

In [ ]:
from skimage import filters

def sub_imgs(collection, img_i_list):
    sub_img_list = []
    for i in img_i_list:
        sub_img_list.append(
            filters.gaussian(collection[i + 1]) 
            - filters.gaussian(collection[i])
        )
    return sub_img_list

In [ ]:
sub_img_list = sub_imgs(frames, [1, 4, 7])

fig, axes = plot.plot_imgs(
    sub_img_list,
    row_title='Subtracted from Succeeding'
)

# Clip highest and lowest intensities

In [ ]:
def clip_imgs(img_list, low=5, high=95):
    clip_img_list = []
    for img in img_list:
        # Clip top and bottom image intensities 
        # (assign low/high for all values below/above low/high)
        p_low, p_high = np.percentile(img, [low, high])
        img -= p_low
        img[(img < 0.0)] = 0.0
        img = img / p_high
        img[(img > 1.0)] = 1.0
        clip_img_list.append(img)
    return clip_img_list

In [ ]:
clip_img_list = clip_imgs(sub_img_list)

fig, axes = plot.plot_imgs(
    clip_img_list, 
    row_title='Clip Intensities'
)

# Invert image

In [ ]:
def invert_imgs(img_list):
    invert_img_list = []
    for img in img_list:
        img = 1 - img
        invert_img_list.append(img)
    return invert_img_list

In [ ]:
invert_img_list = invert_imgs(clip_img_list)

fig, axes = plot.plot_imgs(
    invert_img_list,
    row_title='Inverted'
)

# Denoise using `skimage.restoration.denoise_tv_chambolle`

In [ ]:
from skimage import restoration

def denoise_imgs(img_list):
    denoise_img_list = []
    for img in img_list:
        img = restoration.denoise_tv_chambolle(img, weight=0.15)
        denoise_img_list.append(img)
    return denoise_img_list

In [ ]:
denoise_img_list = denoise_imgs(invert_img_list)

fig, axes = plot.plot_imgs(
    denoise_img_list, 
    row_title='Denoised'
)

# Threshold and create binary image

In [ ]:
def binarize_imgs(img_list, thresh_val=0.4):
    binarize_img_list = []
    for img in img_list:
        img_mask = np.zeros(img.shape, dtype=int)
        img_mask[(img > thresh_val)] = 1
        binarize_img_list.append(img_mask)
    return binarize_img_list

In [ ]:
binarize_img_list = binarize_imgs(denoise_img_list)

fig, axes = plot.plot_imgs(
    binarize_img_list, 
    row_title='Binarized'
)

# Filter minimum region size

In [ ]:
px.imshow(binarize_img_list[0], color_continuous_scale='viridis')

In [ ]:
from skimage import measure

label_img = measure.label(binarize_img_list[0])
# We have been processing images 1, 4, and 7, so 
# binarize_img_list[0] corresponds to collection[1]
img_label_overlay = color.label2rgb(
    label_img, image=frames[1], bg_label=0
)

px.imshow(img_label_overlay, color_continuous_scale='gray')

In [ ]:
import pandas as pd

props = measure.regionprops_table(
    label_img, properties=('label', 'area', 'bbox')
)
props_df = pd.DataFrame(props)
props_df

In [ ]:
props_df.sort_values('area', ascending=False).head()

In [ ]:
regions = measure.regionprops(label_img)

[region.area for region in regions if region.area > 100]

In [ ]:
from skimage import morphology

def filter_size_masks(mask_list, min_size=100):
    filter_size_img_list = []
    for mask in mask_list:
        labels = morphology.label(mask, connectivity=2)
        regions = measure.regionprops(labels)
        filtered_mask = np.zeros(labels.shape, dtype=int)
        for region in regions:
            if region.area > min_size:
                filtered_mask[(labels == region.label)] = 1
                filter_size_img_list.append(filtered_mask)
    return filter_size_img_list

In [ ]:
filter_size_img_list = filter_size_masks(binarize_img_list)
len(filter_size_img_list)
fig, axes = plot.plot_imgs(
    filter_size_img_list, 
    row_title='Size Filtered'
)

# Overlay region bounding box on raw image

In [ ]:
props_df.sort_values('area', ascending=False).head()

In [ ]:
[region.bbox for region in regions if region.area > 100]

In [ ]:
[region.bbox for region in regions if region.area > 100]

In [ ]:
px.imshow(img_label_overlay, color_continuous_scale='gray')

In [ ]:
def gen_bbox_list(mask_list, min_size=100):
    bbox_list = []
    for mask in mask_list:
        labels = morphology.label(mask, connectivity=2)
        regions = measure.regionprops(labels)
        filtered_mask = np.zeros(labels.shape, dtype=int)
        # Iterate through regions in case there is more than one
        # larger than the minimum size
        minr_list, minc_list, maxr_list, maxc_list = [], [], [], []
        for region in regions:
            if region.area > min_size:
                filtered_mask[(labels == region.label)] = 1
                minr_list.append(region.bbox[0]) 
                minc_list.append(region.bbox[1]) 
                maxr_list.append(region.bbox[2]) 
                maxc_list.append(region.bbox[3])
            # Find the true min and max bounding rows and cols
            minr = min(minr_list)
            minc = min(minc_list)
            maxr = max(maxr_list)
            maxc = max(maxc_list)
        bbox_list.append((minr, minc, maxr, maxc))
    return bbox_list

In [ ]:
bbox_list = gen_bbox_list(filter_size_img_list)

fig, axes = plot.plot_bbox(
    filter_size_img_list,
    bbox_list, 
    row_title='Region bounds on mask'
)

In [ ]:
fig, axes = plot.plot_bbox(
    [frames[1], frames[4], frames[7]],
    bbox_list, 
    row_title='Region bounds on raw data'
)